In [1]:
import sys
import os

from typing import Optional, Union, Tuple
import numpy as np

# add parent dir to be able to import utils file
parent_dir = os.path.abspath('../../')
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

import duqling_py.functions as functions
from duqling_interface import DuqlingInterface
duq_r = DuqlingInterface()

In [7]:
def lhs_array(n, d, seed=None) -> np.array:
    if seed is not None: np.random.seed(seed)
    samples = (np.arange(n)[:, None] + np.random.rand(n, d)) / n
    for i in range(d):
        samples[:, i] = np.random.permutation(samples[:, i])
    return samples

In [3]:
NUM_SAMPLES = 100
all_fnames = duq_r.list_functions().fname
failed_funcs = []
for fname in all_fnames:

    if fname == 'cube3_rotate': # currently broken
        continue

    func_info = duq_r.get_function_info(fname)
    input_dim = int(func_info['input_dim'][0])
    input_dim = 9 if fname == 'dts_sirs' else input_dim
    X = lhs_array(NUM_SAMPLES, input_dim)
    X_after, y_r = duq_r.generate_data(fname, 1, X.copy())

    assert (X == X_after).all() # make sure X isn't being mutated by generate_data

    duq_py_func = getattr(functions, fname)
    y_r = duq_r.generate_data(fname, X=X)[1]
    y_p = np.array(list(map(duq_py_func, X)))

    try:
        assert np.isclose(y_r, y_p).all()  # output array shapes should match and values should be more or less identical
    except:
        failed_funcs.append(fname)

/Users/reidmorris/Desktop/duqling/duqling_venv/lib/python3.11/site-packages/numpy/_core/numeric.py:353: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(a, fill_value, casting='unsafe')
/Users/reidmorris/Desktop/duqling/duqling_py/functions/dts_sirs.py:22: RuntimeWarning: overflow encountered in scalar add
  S[t] = S[t-1] + births - deaths[0] - infect + resusc
/Users/reidmorris/Desktop/duqling/duqling_py/functions/dts_sirs.py:22: RuntimeWarning: overflow encountered in scalar subtract
  S[t] = S[t-1] + births - deaths[0] - infect + resusc


In [9]:
failed_funcs

['simple_machine',
 'vinet',
 'ocean_circ',
 'pollutant',
 'beam_deflection',
 'simple_machine_cm',
 'stochastic_piston',
 'cantilever_D',
 'cantilever_S',
 'dts_sirs',
 'ignition']

In [17]:
df = duq_r.list_functions()
df[df.fname.isin(failed_funcs)]

,fname,input_dim,input_cat,response_type,stochastic
24,simple_machine,3.0,False,func,n
25,vinet,3.0,False,func,n
26,ocean_circ,4.0,False,uni,y
29,pollutant,4.0,False,func,n
31,beam_deflection,5.0,False,func,n
35,simple_machine_cm,5.0,False,func,n
36,stochastic_piston,5.0,False,uni,y
37,cantilever_D,6.0,False,uni,n
38,cantilever_S,6.0,False,uni,n
48,dts_sirs,9.0,False,func,y


In [43]:
fname = 'ignition'

func_info = duq_r.get_function_info(fname)
input_dim = int(func_info['input_dim'][0])
X = lhs_array(NUM_SAMPLES, input_dim)
X_after, y_r = duq_r.generate_data(fname, 1, X.copy())

duq_py_func = getattr(functions, fname)
y_r = duq_r.generate_data(fname, X=X)[1]
y_p = np.array(list(map(duq_py_func, X)))

np.isclose(y_r, y_p.transpose()).all()

np.False_